In [1]:
import Pkg
Pkg.add("DifferentialEquations")

    Updating registry at `/opt/julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.8/Project.toml`
  No Changes to `/opt/julia/environments/v1.8/Manifest.toml`


In [2]:
Pkg.add("Plots")

   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.8/Project.toml`
  No Changes to `/opt/julia/environments/v1.8/Manifest.toml`


In [3]:
include("src/Sim.jl")

Main.Sim

In [ ]:
using Pkg
Pkg.add(url="https://github.com/JuliaMBD/JuliaMBDv1.git")

In [ ]:
import Pkg
Pkg.add("Plots")
Pkg.add("DifferentialEquations")

In [ ]:
using JuliaMBDv1
using DifferentialEquations
using Plots

In [ ]:
#hm_bd_r_pntとhm_hd_r_pntはgotoブロックのせいでできていない

In [ ]:
#initialcondition = 6と勝手に指定,掛け算割り算は直接演算,goro不明
@model hm_bd_r_pnt begin
    @parameter g M_Rr_body
    
    @blk In1 = InBlock()
    @blk In2 = InBlock()
    @blk In3 = InBlock()
    @blk Constant1 = ConstantBlock(g)
    @blk Constant = ConstantBlock(M_Rr_body)
    @blk Add2 = AddBlock([:+, :+, :+, :-]) inport[1]:in1 inport[2]:in2 inport[3]:in3 inport[4]:abc
    @blk Integrator = IntegratorBlock(initialcondition = 6)
    @blk Out1 = OutBlock()
    @blk Out2 = OutBlock()
    @blk Out3 = OutBlock()
    
    @blk Goto1 = GotoBlock() #不明
    @blk Goto = GotoBlock() #不明
    
    
    @connect In1 => in1
    @connect In2 => in2
    @connect In3 => in3
    @connect Constant1*Constant => abc
    @connect Add2 / Constant => Goto1
    @connect Add2 / Constant => Integrator1
    @connect Integrator => Out1
    @connect Integrator => Out2
    @connect Integrator => Out3
    @connect Integrator => Goto
end

In [ ]:
@model TestHm_bd_r_pnt begin
    
    @parameter g M_Rr_body
    
    @blk Ramp1 = RampBlock(starttime = 10, slope = 5) outport:inin1
    @blk Ramp2 = RampBlock(starttime = 10, slope = 15) outport:inin2
    @blk hm_b_r = hm_bd_r_pnt(g=g, M_Rr_body=M_Rr_body) inport[1]:r inport[2]:y outport[1]:omega outport[2]:beta  
    @connect inin1 => r
    @connect inin2 => y

end

In [ ]:
m = TestHm_bd_r_pnt(g=9.8, M_Rr_body=44.01);
#sol = @simulate(m, tspan=(0.0, 100.0), scope=(omega));
sol = @simulate(m, tspan=(0.0, 100.0), scope=(omega,beta));
sol.graph


#scopeが1変数の時できていない

In [ ]:
#出来ていない

In [ ]:
#initialcondition = 6と勝手に指定,掛け算割り算は直接演算,outblockを一つ増やした,@blk Constant2 = ConstantBlock(C)はIntegratorBlock()の入力ポートが一つになるように削除した
@model hm_vs_f_pnt begin
    #@parameter z_kz_Fr_body_organs_ini = 0.001526
    @parameter kz_Fr_body_organs dz_Fr_body_organs M_Fr_body_organs g
    
    @blk In = InBlock()
    @blk Add1 = AddBlock([:-, :+]) inport[1]:inte inport[2]:in1
    #@blk Constant2 = ConstantBlock(C)
    @blk Integrator1 = IntegratorBlock(initialcondition = 6)
    @blk Gain2 = GainBlock(kz_Fr_body_organs)
    @blk Gain3 = GainBlock(dz_Fr_body_organs)
    @blk Add4 = AddBlock([:+, :+]) inport[1]:gain2 inport[2]:gain3
    @blk Gain1 = GainBlock(-1)
    @blk Constant = ConstantBlock(M_Fr_body_organs)
    @blk Constant1 = ConstantBlco(g)
    @blk Add = AddBlock([:+, :-]) inport[1]:add4 inport[2]:pro
    @blk Integrator = IntegratorBlock(initialcondition = 6)
    @blk Out1 = OutBlock
    
    @blk Out2 = OutBlock(M_Fr_body_organs)

    
    
    @connect In => in1
    @connect Integrator => inte
    @connect Add1 => Integrator1
    @connect Integrator1 => Gain2
    @connect Add1 => Gain3
    @connect Gain2 => gain2
    @connect Gain3 => gain3
    @connect Add4 => add4
    @connect Constant1 * Constant => pro
    @connect Add / Constant => Integrator
    @connect Add4 => Gain1
    
    @connect Gain1 => Out1
    @connect Gain2 => Out2
    
    
end

In [ ]:
@model TestHm_vs_f_pnt begin
    
    @parameter kz_Fr_body_organs dz_Fr_body_organs g M_FR_body
    
    @blk Ramp1 = RampBlock(starttime = 10, slope = 5) outport:inin1
    #@blk Ramp2 = RampBlock(starttime = 10, slope = 5) outport:inin2
    #@blk hm_v = hm_vs_f_pnt(g=g, M_FR_body=M_FR_body) inport[1]:r inport[2]:y outport[1]:omega outport[2]:beta
    @blk hm_v = hm_vs_f_pnt(kz_Fr_body_organs=kz_Fr_body_organs, dz_Fr_body_organs=dz_Fr_body_organs, M_Fr_body_organs=M_Fr_body_organs, g=g) inport:r outport[1]:omega outport[2]:beta
    @connect inin1 => r

end

In [ ]:
m = TestHm_vs_f_pnt(kz_Fr_body_organs=82200 dz_Fr_body_organs=195 M_Fr_body_organs=12.8 g=9.8);
#sol = @simulate(m, tspan=(0.0, 100.0), scope=(omega));
sol = @simulate(m, tspan=(0.0, 100.0), scope=(omega,beta));
sol.graph


#scopeが1変数の時できていない

In [ ]:
#集合ブロックHM_F_PNTはhm_bd_f_pntとhm_hd_f_pntブロックが作成困難のため出来ない